### Trabalho 1

Seguem instruções:

1) Baixar dataset em : https://www.kaggle.com/c/home-credit-default-risk/data

2) Explorar, conforme instruções:

* 'application_train.csv': Esse csv possui 122 colunas: explorar 40 colunas, sendo as primeiras 30 obrigatórias e 10 adicionais a escolher
* 30 colunas obrigatórias:

['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE',
       'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS',
       'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE',
       'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE',
       'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS']
       
* alunos deverão se organizar para que todas as colunas sejam exploradas
* escolher dois arquivos, para exploração e join/merge/concat com o 'application_train.csv'

['HomeCredit_columns_description.csv',
 'POS_CASH_balance.csv',
 'credit_card_balance.csv',
 'installments_payments.csv',
 'application_train.csv',
 'bureau.csv',
 'previous_application.csv',
 'bureau_balance.csv',
 'sample_submission.csv']
 
3) objetivos:
* entendimento do problema de negócio
* carregamento do dataset
* identificação e tratamento de NaN
* exploração univariadaj, bivariada
* visualizações
* insights de negócio
* formatação do dataset para algoritmos de machine learning
* criação de features: agregação de dados, junção de dois ou mais dataframes

4) entregável: jupyter notebook. Não será necessário apresentação em powerpoint, nem relatório.

5) entrega limite: 8/agosto/2018 

6) trabalho individual. Haverá aplicação sistema anti-plágio nas submissões. (http://www.fgv.br/mailing/codigo-etica-conduta/24/)
   suspeitas de plágio estarão sujeitos a arguição oral de 5 mins por aluno no ultimo dia de aula (11/agosto/2018),

7) avaliação:
   - jupyter notebook e arguição (se houver)

### Problema de Negócio

##### Do Kaggle:
Many people struggle to get loans due to insufficient or non-existent credit histories. And, unfortunately, this population is often taken advantage of by untrustworthy lenders.

Home Credit strives to broaden financial inclusion for the unbanked population by providing a positive and safe borrowing experience. In order to make sure this underserved population has a positive loan experience, Home Credit makes use of a variety of alternative data--including telco and transactional information--to predict their clients' repayment abilities.

While Home Credit is currently using various statistical and machine learning methods to make these predictions, they're challenging Kagglers to help them unlock the full potential of their data. Doing so will ensure that clients capable of repayment are not rejected and that loans are given with a principal, maturity, and repayment calendar that will empower their clients to be successful.

##### Entendimento do problema:

Como fornecer crédito para pessoas com histórico insuficiente ou inexistente?

Utilizando estes dados alternativos, pode ser criado um modelo preditivo quer será usado para fazer uma análise das pessoas, para saber se elas tem um histórico de bom pagamento, e fornecer crédito para estas pessoas.

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

import matplotlib.pyplot as plt
import missingno as mn

import seaborn as sns

pd.__version__



In [ ]:
df = pd.read_csv('application_train.csv', index_col=0)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info(verbose=True,null_counts=True)

In [ ]:
mn.bar(df,figsize=(40,6))

In [ ]:
df.corr()

In [ ]:
plt.matshow(df.corr())

In [ ]:
# Generate a mask for the upper triangle
mask = np.zeros_like(df.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(df.corr(), mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
df.iloc[:,0:40].head()

In [ ]:
df40 = df.iloc[:,0:40]

In [ ]:
df40.to_csv('application_train_40cols.csv')

In [ ]:
df40 = pd.read_csv('application_train_40cols.csv')

In [ ]:
df40.head()

In [ ]:
df40.info()

##### Analisando as variáveis relativas a CARROS e removendo NaN de OWN_CAR_AGE baseado na FLAG_OWN_CAR

Foi observado que quando a pessoa não possuia carro ('FLAG_OWN_CAR'='N') o valor de 'OWN_CAR_AGE'=NaN 

In [ ]:
df40.loc[:,['FLAG_OWN_CAR','OWN_CAR_AGE']].head(20)

Substituindo NaN por 0, para indicar que não existe carro

In [ ]:
df40.OWN_CAR_AGE.fillna(0, inplace = True)

In [ ]:
df40.loc[:,['FLAG_OWN_CAR','OWN_CAR_AGE']].head(20)

In [ ]:
df40.loc[:,['FLAG_OWN_CAR','OWN_CAR_AGE']].info()

##### Analisando as variáveis relativas as familias

In [ ]:
df40.loc[:,['CNT_CHILDREN',lambda df40: df40['NAME_FAMILY_STATUS'] == 'Unknown','CNT_FAM_MEMBERS']]

In [ ]:
set(df40.NAME_FAMILY_STATUS)

Os diferentes valores que temos em Family status nos ajudam a definir um valor para este status:

{'Civil marriage' = 2,
 'Married' = 2,
 'Separated' = 1,
 'Single / not married' = 1,
 'Unknown' = CNT_FAM_MEMBERS - CNT_CHILDREN,
 'Widow' = 1}

In [ ]:
df40.NAME_FAMILY_STATUS.loc[df40.NAME_FAMILY_STATUS == 'Civil marriage'] = 2
df40.NAME_FAMILY_STATUS.loc[df40.NAME_FAMILY_STATUS == 'Married'] = 2
df40.NAME_FAMILY_STATUS.loc[df40.NAME_FAMILY_STATUS == 'Separated'] = 1
df40.NAME_FAMILY_STATUS.loc[df40.NAME_FAMILY_STATUS == 'Single / not married'] = 1
df40.NAME_FAMILY_STATUS.loc[df40.NAME_FAMILY_STATUS == 'Widow'] = 1

for i in df40.index:
    if df40.NAME_FAMILY_STATUS[i] == 'Unknown': 
        df40.NAME_FAMILY_STATUS[i] = (df40.CNT_FAM_MEMBERS[i] - df40.CNT_CHILDREN[i])

In [ ]:
#df40.NAME_FAMILY_STATUS_NO

df40.loc[:,['CNT_CHILDREN','NAME_FAMILY_STATUS','CNT_FAM_MEMBERS']].head(20)

In [ ]:
df40.loc[:,['CNT_CHILDREN','NAME_FAMILY_STATUS','CNT_FAM_MEMBERS']].info()

In [ ]:
mn.matrix(df40)

In [ ]:
pd.isnull(df40).any(1).nonzero()[0]

In [ ]:
df40.loc[[41982,187348],['CNT_CHILDREN','NAME_FAMILY_STATUS','CNT_FAM_MEMBERS']]

In [ ]:
mn.bar(df40,figsize=(40,6))

Considerando que somente as duas linhas da familia estao com valores faltantes, podemos ignorar estes valores, apagando estas duas linhas do dataframe

In [ ]:
df40.NAME_FAMILY_STATUS.dropna(inplace=True)

In [ ]:
pd.isnull(df40).any(1).nonzero()[0]

In [ ]:
mn.bar(df40,figsize=(40,6))

In [ ]:
df40.to_csv('application_train_40cols_T.csv')

#### Juntando dataframes para análise

In [ ]:
pd.read_csv('bureau_balance.csv', index_col=0).head()

Assumindo que 0-40 satisfaz o trabalho, voltamos para o dataframe completo, para análises complementares.

# Dataframe completo

In [ ]:
df = pd.read_csv('application_train.csv', index_col=0)

In [ ]:
df.info(verbose=True,null_counts=True)

In [ ]:
mn.bar(df,figsize=(40,6))

Aplicando as soluções para Carro e Familia, porém sem o dropna

In [ ]:
#Para o carro, podemos substituir o NA por 0
df.OWN_CAR_AGE.fillna(0, inplace = True)

In [ ]:
mn.bar(df,figsize=(40,6))

Salvando status com as variáveis de CARROS tratado

In [ ]:
df.to_csv('application_train_T1.csv')

lendo novamente o arquivo para continuar a limpeza

In [ ]:
df = pd.read_csv('application_train_T1.csv', index_col=0)

In [ ]:
#Usamos a mesma logica para a familia
df['AMT_FAMILY_PEOPLE'] = 0
df.AMT_FAMILY_PEOPLE.loc[df.NAME_FAMILY_STATUS == 'Civil marriage'] = 2
df.AMT_FAMILY_PEOPLE.loc[df.NAME_FAMILY_STATUS == 'Married'] = 2
df.AMT_FAMILY_PEOPLE.loc[df.NAME_FAMILY_STATUS == 'Separated'] = 1
df.AMT_FAMILY_PEOPLE.loc[df.NAME_FAMILY_STATUS == 'Single / not married'] = 1
df.AMT_FAMILY_PEOPLE.loc[df.NAME_FAMILY_STATUS == 'Widow'] = 1

for i in df.index:
    if df.NAME_FAMILY_STATUS.loc[i] == 'Unknown':
        df.AMT_FAMILY_PEOPLE.loc[i] = (df.CNT_FAM_MEMBERS.loc[i] - df.CNT_CHILDREN.loc[i])

In [ ]:
mn.bar(df,figsize=(40,6))

In [ ]:
df.info(verbose=True,null_counts=True)

Tratando as variáveis de Emprego

In [ ]:
df.loc[:,['DAYS_EMPLOYED','FLAG_WORK_PHONE','OCCUPATION_TYPE']].head()

## Sorteio de colunas para o DataFrame

In [ ]:
df = pd.read_csv('application_train.csv', index_col=0)

In [ ]:
semente = 2701 #Data de aniversário
df_temp=df.iloc[:,31:]
df40 = pd.concat([df.iloc[:,0:30], df_temp.sample(10,axis=1,random_state=semente)], axis=1)

In [ ]:
df40.info()